In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from konlpy.tag import Kkma,Twitter
from konlpy.utils import pprint
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

CUDA_VISIBLE_DEVICES = 1,2
print(torch.cuda.current_device())

gpu_num = 2;

use_cuda = torch.cuda.is_available()
kkma = Kkma()
twitter = Twitter()
MAX_LENGTH = 80

0


In [2]:
#make dict

SOS_token = 0
EOS_token = 1

class Lang :
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {}
        self.word2count = {0: "SOS", 1: "EOS"}
        self.n_words = 2 #count SOS and EOS and UNKWON
        
    def addSentence(self, sentence):
        for word in twitter.morphs(sentence): #kkma.morphs(sentence): # 형태소 단위로 단어 자르기
            #print(word)
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
#Turn a Unicode stirng to plain ASCII

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^a-zA-Z.!? 0-9 \t]+')
    result = hangul.sub('', s)

    
    return result

In [4]:
def readText():
    print("Reading lines...")
    
    inputs = open('../data/context/hangul.txt', encoding='utf-8').read().strip().split('\n')

    inputs = [normalizeString(s) for s in inputs]

    words = Lang('words')
    
    triples = []
    
    temp = [s.split('\t') for s in inputs]
    print(temp[0])
    print("Counting words...")
    for i in range(0, len(temp)-2, 2):
        words.addSentence(temp[i][-1])
        words.addSentence(temp[i+1][-1])
        words.addSentence(temp[i+2][-1])
        context = []
        if temp[i][0] == temp[i+1][0] and temp[i+1][0] == temp[i+2][0]:
            for j in range(3):
                context.append(temp[i+j][-1].strip())      
        if context:
            triples.append(context)
    
    print("words length : ", words.n_words)
        
#     context = Lang('context')
#     message = Lang('message')
#     response = Lang('response')
    
    return words , triples #context, message, response, triples


In [5]:
def prepareData():
    words, triples = readText() #context_lang, message_lang, response_lang, triples = readText()
    print("Read %s sentence pairs" % len(triples))

#     print("Counting words...")
#     for triple in triples:
#         words.addSentence(triple[0])
#         words.addSentence(triple[1])
#         words.addSentence(triple[2])
        
    print("Counted words:")
    print(words.name, words.n_words)
#     print(context_lang.name, context_lang.n_words)
#     print(message_lang.name, message_lang.n_words)
#     print(response_lang.name, response_lang.n_words)
    
    return words, triples

words, triples = prepareData()

print(random.choice(triples))

Reading lines...
['1', '1', ' 안녕하십니까?']
Counting words...
words length :  20373
Read 40979 sentence pairs
Counted words:
words 20373
['네.', '제가 먹고 있던 요리에서 머리카락이 나왔습니다. 이게 어떻게 된 일이지요?', '정말 죄송합니다.']


In [6]:
print(random.choice(triples))

['무엇을 찾으세요?', '급하게 빈방을 찾는데요. 방 하나 있나요?', '작은 독방 한 개가 남아있습니다.']


In [7]:
def indexesFromSentence(lang, sentence):
    ret = []
    for word in twitter.morphs(sentence):#kkma.morphs(sentence):#sentence.split(' '):
        ret.append(lang.word2index[word])
    return ret
#     return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1,1))
    if use_cuda:
        return result.cuda(gpu_num)
    else:
        return result

def variablesFromTriple(triple):
    context_variable = variableFromSentence(words, triple[0])
    message_variable = variableFromSentence(words, triple[1])
    response_variable = variableFromSentence(words, triple[2])
    return (context_variable, message_variable, response_variable)

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
        #self.lstm = nn.LSTM(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        
        output = embedded
#         output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.relu(output)
        #output, hidden = self.gru2(output, hidden)
#         output = F.relu(output)
        #output, hidden = self.lstm(output, hidden)
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1,1, self.hidden_size))
        if use_cuda:
            return result.cuda(gpu_num)
        else:
            return result

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2 , self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        
        self.gru2 = nn.GRU(self.hidden_size, self.hidden_size)
        #self.lstm = nn.LSTM(self.hidden_size, self.hidden_size,2)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1,1,-1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]) , 1 )))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        #output, hidden = self.lstm(output)
        output, hidden = self.gru(output, hidden)
        output = F.relu(output)
        output, hidden = self.gru2(output, hidden)
        
        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights
    
    def initHidden(self):
        result = Variable(torch.zeros(1,1,self.hidden_size))
        if use_cuda:
            return result.cuda(gpu_num)
        else:
            return result
        

In [10]:
teacher_forcing_ratio = 0.5


def train(context_variable, message_variable, response_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    context_length = context_variable.size()[0]
    message_length = message_variable.size()[0]
    response_length = response_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda(gpu_num) if use_cuda else encoder_outputs
    
    input_variable = torch.cat((context_variable,message_variable), 0)
    input_length = input_variable.size()[0]
    
    loss = 0
    #print(input_length)
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda(gpu_num) if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(response_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, response_variable[di])
            decoder_input = response_variable[di]

    else:
        for di in range(response_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda(gpu_num) if use_cuda else decoder_input

            loss += criterion(decoder_output, response_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / response_length

In [11]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m*60
    return '%dm %ds' % (m,s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [12]:
def trainIters(encoder, decoder , n_iters, print_every=2000, plot_every= 50, learning_rate=0.01):
    start = time.time()
#     plot_losses = []
#     print_loss_total = 0
#     plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    train_len = int(float(len(triples))*0.15)
    
    training_tris = [variablesFromTriple(triples[i]) for i in range(train_len)]
    random.shuffle(training_tris)
    
    criterion = nn.NLLLoss()
    
    for epoch in range(1, n_iters + 1):
        print('epoch : ' , epoch)
        print_losses = []
        print_loss_total = 0
        plot_loss_total = 0
        for i in range(train_len):
            training_tri = training_tris[i]
            context_variable = training_tri[0]
            message_variable = training_tri[1]
            response_variable = training_tri[2]

            loss = train(context_variable, message_variable, response_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if (i + 1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%d : %.4f' % (i+1, print_loss_avg))

#             if i % plot_every == 0:
#                 plot_loss_avg = plot_loss_total / plot_every
#                 plot_losses.append(plot_loss_avg)
#                 plot_loss_total = 0
        
        #showPlot(plot_losses)

In [13]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [14]:
def evaluate(encoder, decoder, context, message, max_length=MAX_LENGTH):
    context_variable = variableFromSentence(words, context)
    message_variable = variableFromSentence(words, message)
    encoder_hidden = encoder.initHidden()
    
    input_variable = torch.cat((context_variable, message_variable),0)
    input_length = input_variable.size()[0]
    
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda(gpu_num) if use_cuda else encoder_outputs
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]
        
    decoder_input = Variable(torch.LongTensor([[SOS_token]])) #SOS
    decoder_input = decoder_input.cuda(gpu_num) if use_cuda else decoder_input
    
    decoder_hidden = encoder_hidden
    
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder( decoder_input, decoder_hidden, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(words.index2word[ni])
        
        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda(gpu_num) if use_cuda else decoder_input
        
    return decoded_words, decoder_attentions[:di +1]

In [18]:
def evaluateTestSet(encoder, decoder, n=10):
    for i in range(int(float(len(triples))*0.2),int(float(len(triples))*0.25)):
        triple = random.choice(triples)
        print('>', triple[0])
        print('>', triple[1])
        print('=', triple[2])
        output_words, attentions = evaluate(encoder, decoder , triple[0], triple[1])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [16]:
hidden_size = 512
encoder1 = EncoderRNN(words.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, words.n_words, dropout_p=0.1)


if use_cuda:
    encoder1 = encoder1.cuda(gpu_num)
    attn_decoder1 = attn_decoder1.cuda(gpu_num)
    
trainIters(encoder1, attn_decoder1, 20, print_every=500)

epoch :  1
500 : 5.0806
1000 : 4.7411
1500 : 4.6620
2000 : 4.6166
2500 : 4.5145
3000 : 4.3832
3500 : 4.3722
4000 : 4.2729
4500 : 4.2580
5000 : 4.2701
5500 : 4.1449
6000 : 4.2915
epoch :  2
500 : 4.2687
1000 : 4.1134
1500 : 4.3030
2000 : 4.1273
2500 : 4.0559
3000 : 3.9140
3500 : 3.9467
4000 : 4.0757
4500 : 3.8891
5000 : 4.0398
5500 : 3.8023
6000 : 4.0071
epoch :  3
500 : 3.9330
1000 : 3.8138
1500 : 3.8958
2000 : 3.9714
2500 : 3.8699
3000 : 3.7498
3500 : 3.8256
4000 : 3.8351
4500 : 3.8553
5000 : 3.8287
5500 : 3.6019
6000 : 3.8084
epoch :  4
500 : 3.7828
1000 : 3.7662
1500 : 3.7220
2000 : 3.8306
2500 : 3.7759
3000 : 3.6092
3500 : 3.6386
4000 : 3.6739
4500 : 3.6603
5000 : 3.6124
5500 : 3.5479
6000 : 3.5828
epoch :  5
500 : 3.5998
1000 : 3.5107
1500 : 3.6498
2000 : 3.7275
2500 : 3.5420
3000 : 3.4955
3500 : 3.5013
4000 : 3.5146
4500 : 3.4519
5000 : 3.5149
5500 : 3.3193
6000 : 3.4154
epoch :  6
500 : 3.5173
1000 : 3.3373
1500 : 3.4503
2000 : 3.5326
2500 : 3.3611
3000 : 3.3100
3500 : 3.2737
40

In [19]:
evaluateTestSet(encoder1, attn_decoder1)

> 아니면 가까운 약국에 약 처방을 받고 싶은데요.
> 약 처방은 의사만 내릴 수 있기 때문에 병원을 먼저 가셔야 합니다.
= 그럼 어느 병원으로 가야 하나요?
< 후문 은 참 없네 데 요 참 은 어떻 게 알 고 여행 좋 . . <EOS>

> 여기 빵 토스기를 이용하는 법이 어떻게 되나요?
> 빵을 넣고 타이머를 이렇게 돌려놓아 다 되었을 때 소리가 나면 가져다 먹으면 됩니다.
= 감사합니다. 여기 집게가 없는데요.
< 그럼 그때 은 익히 를 불러 주 시 면 됩 니다 . <EOS>

> 객실로 가져다주세요. 7시 10분에.
> 예 알겠습니다. 모닝콜과 아침 준비해 드리겠습니다.
= 네 감사합니다.
< 감사합 니다 . <EOS>

> 저기요 시청을 가는데 이쪽으로 타면 되나요?
> 예 맞아요.
= 감사합니다. 급하게 탔는데 다행이네요.
< Third Street 을 계속 왔 습니다 . <EOS>

> 예 더 필요하신 건 없으신가요?
> 접시 좀 채워주시고 깨끗한 냅킨 하나 갖다 주세요.
= 예 알겠습니다. 이건 서비스로 드리는 겁니다.
< 예 알 겠 습니다 . <EOS>

> 호텔 전체에 무선 랜이 설치되어 있습니다.
> 노트북으로 말고 일반 컴퓨터가 설치된 곳은 있습니까?
= 지하 1층에 컴퓨터를 무료로 사용할 수 있는 곳이 있습니다.
< 여기 있 습니다 . <EOS>

> 하지만 이 얼룩은 어떻게 하죠?
> 셔츠를 구매하시기 전에 잘 살펴보셔야 했습니다. 죄송합니다.
= 정말 실망스럽네요.
< 괜찮 은 은 카드 니까 ? <EOS>

> 샌드위치와 치킨샐러드가 7달러로 가장 저렴합니다.
> 그럼 샌드위치 갖다 주세요.
= 네. 곧 준비해 드리겠습니다.
< 예 알 겠 습니다 . <EOS>

> 이번 달까지 이 패키지를 구입하시는 분들께는 보스턴백을 선물로 드려요.
> 그거 좋은데요?  이 패키지를 구입할게요.
= 아버님께서 좋아하실 거예요. 계산 도와드리겠습니다.
< 많이 1 는 분 후 에 익혀 주 세요 . <EOS>

> 제가 그쪽으로 보내면 되나요?
> 아

< 얼마 입 니까 ? <EOS>

> 그렇죠. 래프팅을 즐기신 다음 폭포다이빙도 동시에 즐길 수 있습니다.
> 잘됐네요. 좀 더 자세히 설명해 주실 수 있나요?
= 여기 프로그램을 설명해 놓은 책자가 있으니깐 한번 읽어 보세요.
< 여기 있는 카드 가 10 만 이 있 습니다 . <EOS>

> 실례합니다. 제가 아무리 해도 이 수도꼭지의 물이 안 나오네요.
> 수도꼭지 핸들을 잘 당겼나요?
= 제가 이런 수도꼭지는 처음 보는데 이것을 어떻게 사용하는지 보여주시겠어요?
< 아무래도 발목 을 이 계속 됩 니다 . <EOS>

> 잘못해서 해파리를 밟게 된다면 쏘일지도 몰라요.
> 어떻게 나가죠? 무서워서 못 나가겠어요.
= 제가 튜브를 던지겠어요. 잡으셨나요?
< 이 게 이 게 이 어요 ? 요 . <EOS>

> 가장 효능 있는 약을 추천해 주세요.
> 개인적으로는 자연에 가까운 한방 쪽을 추천해 드립니다. 장기적으로 체질을 개선해 주는 효과도 있습니다.
= 그럼 저렴하고 잘 듣는 한방변비약으로 부탁합니다.
< 어디가 길 을 봐 요 어떻 게 되 나요 ? <EOS>

> 15분 정도 걸릴 것 같습니다.
> 조금만 서둘러 주세요.
= 바쁘신 일이 있으신가요?
< 네 지금 택시 해주세 있 . <EOS>

> 아 시간이 지났군요. 그러면 다른 세트메뉴가 있나요?
> 빅맥세트와 치즈버거세트가 있습니다.
= 가격은 각각 어떻게 되나요?
< 가격 은 각각 어떻 게 되 나요 ? <EOS>

> 좋습니다. 그걸로 쓰겠습니다.
> 감사합니다.
= 천만에요.
< 별 말씀 을 요 . <EOS>

> 아니요 괜찮습니다.
> 그럼 위치를 알려 드릴까요?
= 네 그 병원이 어디에 있습니까?
< 아니 요 . 요 요 . <EOS>

> 글쎄요 주말이라 다른 기구에도 기다리시는 분들이 많답니다.
> 실외보다 실내가 더 덜 복잡한가요?
= 네 그렇기도 하고 냉방시설이 되어 있어서 기다리시는 데 시원하실 것 같네요.
< 지금 바로 에 . . . <EOS>

> 오늘 택배 올 게 있는데 지금 나가봐야

< 2 이 쪽 등 이 <EOS>

> 대중교통을 이용하시려고 합니까?
> 네 그렇습니다.
= 조금 힘드실 텐데요.
< 신발 은 불량품 인가 보네 요 . 바꿔 드리 겠 습니다 . <EOS>

> 여기 있습니다.
> 10일 것이 맞군요.
= 네 재미있게 보십시오.
< 이 제품 이 마음 에서 맛있 습니다 . <EOS>

> 위치는 알려주셨나요?
> 알려줘도 잘 못 찾네요 원래 길을 잘 못 찾는 친구라서요.
= 그럼 어떻게 해 드리면 되나요?
< 그럼 다른 분 여행 중 에 가 다 를 다 요 . 다 . <EOS>

> 서둘러야 할 것 같네요. 폐점까지 1시간 밖에 남지 않았어요.
> 그렇게 일찍 끝나나요?
= 겨울에는 문을 일찍 닫아요.
< 아니 요 여기 만 변경 서 차 차 를 타고 드리 겠 습니다 <EOS>

> 기스본 정원페스티발에요. 이걸 보시다니 운이 좋으시네요.
> 운이 좋다니 무슨 말씀이신가요?
= 기스본 정원 페스티발은 3년에 한번씩밖에 열리지 않거든요.
< 아니 요 . 가방 을 가 . . . . 한 고 . 요 . . <EOS>

> 나도 거스름돈이 없어요. 잔돈 얼마 가지고 계신데요?
> 잠시만요. 여기 앉아서 동전 좀 모아 볼게요.
= 네 천천히 하세요.
< 여기 게 안 되 나요 ? <EOS>

> 네 무엇을 도와드릴까요?
> 길을 잃어버린 것 같아서요.
= 어디로 가시죠?
< 버스 번호 는 잘못 번호 가 주 있 . <EOS>

> 네. 카메라가 참 좋군요.
> 여행을 위해 새로 장만하였습니다. 감사합니다.
= 어떤 버튼을 누르면 되나요?
< 그럼 다른 . 은 은 <EOS>

> 카르카손에 가는 게 어떻습니까? 유네스코 세계문화유산으로도 지정되어 있습니다.
> 그런 곳이 있었군요. 어떻게 가는지 아십니까?
= 오드 강을 중심으로 동서로 나뉘어 있는데 동쪽으로 가야 합니다.
< 이 모르 겠 어요 . 가방 . ? 주문 도 와 드리 겠 습니다 . <EOS>

> 메트로버스를 타시면 1달러 25센트입니다.
> 이곳은 환승이 가능하나요?
= 아니요 환승이 없어서

< 쥬스 가 가 정도 가 정도 가 봐 요 . <EOS>

> 오늘 추천할만한 음식은 무엇이죠?
> 저희 식당은 국수 전문집입니다.
= 가격이 얼마나 되지요?
< 이 있 그럼 요 다 . 요 . 다 . . <EOS>

> 그리고 제 가방에서 옷을 꺼내는 것을 좀 도와주시겠습니까?
> 네 가방이 어떤 것입니까?
= 까만색 미니 트렁크입니다.
< 블루 곱창 입니 다 . 가방 을 갖다 겠 어요 ? 가방 이 요 ? <EOS>

> 어제는 못 봤는데 뒷부분이 좀 뜯어져 있더라고요.
> 그렇군요. 영수증을 가지고 오셨나요?
= 네 혹시 이것도 환불할 수 있나요? 옷이 생각보다 별로라서요.
< 손님 차 저희 가 아니 면 면 면 면 됩 니다 . 됩 <EOS>

> 밀러 호가든 코로나.
> 호가든 주세요.
= 알겠습니다. 신분증 좀 확인할까요.
< 혹시 모르니 카드 있 빨리 주 세요 주 세요 . <EOS>

> 제가 닭 알러지가 있어서 그 요리는 못 먹을 것 같습니다. 다른 것은 없나요?
> 다른 음식으로는 쇠고기 안심스테이크 요리가 있습니다.
= 좋습니다. 그걸로 주세요.
< 길 을 가 봐 도 될 까 요 ? <EOS>

> 원하시는 메뉴를 방에 비치한 메뉴카드에 체크해주십시오.
> 네 메뉴카드는 체크한 후 어디에 둬야 되나요?
= 새벽 2시까지 현관 바깥쪽 문고리에 걸어주시면 확인됩니다.
< 네 버스 는 은 . <EOS>

> 여행 기간이 어떻게 되시나요?
> 10박 11일 일정이니 11일간입니다.
= 성인 두 분 다 가입하시는 거죠?
< 성인 두 분 다 거 <EOS>

> 중식과 한식 모두 67달러 정도 합니다.
> 감사합니다.
= 예 즐거운 식사 되시기 바랍니다.
< 별 말씀 을 요 . 편안한 여행 되 세요 . <EOS>

> 이 종이에 오늘 하루 동안 동선과 의심구간을 적어주세요.
> 네. 이제 어떻게 해야 하나요?
= 여권이 없으시니 일단 대사관에 연락을 하세요.
< 네 이 다음 역 에서 내려 테 테 열차 . 열차 에서 내려 드리 겠 습니다 . <EOS>

> 물건을 구

< 이 를 이 많이 를 메뉴 를 로 있 습니까 ? 요 . <EOS>

> 조금 있으면 목적지에 도착합니다.
> 그런데 왜 이 길로 가시나요?
= 무슨 말씀이신지요?
< 이 는 변경 변경 변경 변경 변경 변경 을 변경 을 변경 을 하고 있 습니다 . <EOS>

> 어디로 가야 하나요?
> 앞쪽으로 두 칸 더 가시면 식당이 나옵니다.
= 음식의 맛은 어떻습니까?
< 이 로 있 나요 ? 요 이 은 은 있 나요 ? <EOS>

> 혹시 얼음을 주실 수 있습니까?
> 네 얼음 한 컵에 20센트씩 판매하고 있습니다.
= 얼음 두 컵만 가져다주세요.
< 우회 감사합 니다 . 그럼 뒤 에 대해 겠 네요 . <EOS>

> 네 금방 준비해 드리겠습니다.
> 감사합니다. 여기 화장실이 어디에 있죠?
= 입구 오른쪽에 있습니다.
< 라스베가스 로 가세 요 . <EOS>

> 그렇다면 원하시는 것이 편도 항공편이십니까?
> 아니요. 왕복티켓으로 1장 부탁드립니다.
= 언제 돌아오실 예정이십니까?
< 20 분 정도 기다려 으로 지금 후 <EOS>

> 성공회 가톨릭 개신교 등 종류가 많잖아요.
> 네 가톨릭 교회를 찾고 싶은데요.
= 중앙 사거리에서 왼쪽으로 돌아가면 높은 십자가가 보이는데 그게 성공회 교회고요. 거기서 한 100미터쯤 더 가면 가톨릭 교회예요.
< 중앙 건물 들 을 만 들 이 사람 들 들 만 지금 은 들 들 들 들 의 에서 . 있어 요 고 사람 들 들 과 사람 들 들 들 요 . 사람 들 들 과 사람 들 들 은 들 들 들 들 들 들 들 요 . <EOS>

> 이태원으로 가주세요.
> 네 알겠습니다.
= 그런데 이 택시는 기본요금이 더 비싼가요?
< 몇 분 이 들 은 은 가 가 가 가 가 오 . <EOS>

> 그런데 열흘 다 묵으실 거면 5일치는 미리 선불로 주셨으면 좋겠네요.
> 그래요? 지금 가져다 드릴게요.
= 고맙습니다. 보통 장기 투숙객들은 저희가 그렇게 원칙을 세워서요.
< 예 갖다 주시 은 카드 하나요 ? 까 요 ? <EOS>

> 시간당 800원이고 

...

< 네 감사합 니다 . <EOS>

> 화물차가 빙판길에 미끄러져 다른 차들과 추돌사고가 있었던 것 같습니다.
> 이런. 큰 사고가 났군요.
= 네 큰 사고라서 사고를 정리하기 위해 많은 시간이 걸리는 것 같습니다.
< 네 바로 바로 분들 의 기차 가 있었 어요 . <EOS>

> 이탈리아는 인터넷이 조금 느립니다. 혹시 노트북 가지고 있습니까?
> 제 것이 있습니다. 제 컴퓨터로 연결해서 사용하겠습니다.
= 로비 우측 소파 옆에 컴퓨터가 있습니다.
< 로비 우측 로비 가 있 습니다 . 저희 가 에서 . 겁니 다 . 요 . <EOS>

> 수영복을 챙겨오길 정말 잘한 것 같아요.
> 수영장이 작았으면 울었을 것 같네요.
= 네 아마 그랬을 거예요. 기대를 많이 했거든요.
< 유리공예 가 적어 둔 것 이 이 죠 ? <EOS>

> 담당 웨이터를 부르겠습니다. 이 사람 맞나요?
> 예 맞아요. 저희 기억하시죠?
= 아까 좀 더 구워달라고 하셨잖아요?
< 이 거 죠 지 . <EOS>

> 음식 나왔습니다. 세팅해 드리겠습니다.
> 감사합니다.
= 시싱은 어디로 드릴까요?
< 별 말씀 요 . 가 있으 니 가서 식사 가 식사 할 게요 . <EOS>

> 피해는 얼마나 되십니까?
> 차 뒤 유리창과 범퍼가 깨졌습니다.
= 여기 이 서류를 작성해주십시오.
< 감사합 니다 . <EOS>

> 예약된 기일이 지나지 않았는데요.
> 네 사정이 생겨서 예정보다 빨리 가게 되었습니다.
= 그렇군요. 확인하고 체크아웃해 드리겠습니다.
< 네 지금 은 가 가 것 들 고 를 요 를 . <EOS>

> 감사합니다. 그런데 제 구두는 이 백화점에서 산 것이 아니에요. 저는 지금 여행 중이거든요.
> 네 저희 백화점에서는 저희 백화점에서 구매하신 것이 아니더라도 수선해 드리고 있습니다.
= 정말요? 다행이네요. 그런데 무료인가요?
< 그럼 죽 을 계속 후 에 맞춰 가세 요 . <EOS>

> 정방폭포로 가주세요.
> 제주도는 처음이신가요?
= 여러 번 왔는데 올 때마다 새롭고 좋네요.
< 제

< 여기 있 습니다 . 가격 이 습니까 ? 것 . <EOS>

> 걸어서 가실 겁니까?
> 걸어서 오래가야 합니까?
= 걸어서 가신다면 15분 정도 소요됩니다.
< 아니 요 . <EOS>

> 세트는 더 저렴한가요?
> 네.
= 얼마 정도 저렴한가요?
< 그럼 이 가 가 가 요 많이 오시 게요 . <EOS>

> 그럼 매운탕으로 주문하겠습니다.
> 몇 인분으로 주문하시겠습니까?
= 3인분 해 주십시오.
< 2 번 이 로 있 . . . <EOS>

> 여기에서요? 한 개 고르면 되나요?
> 네 골라보세요.
= 이게 좋겠네요. 아주 예쁘네요. 감사합니다.
< 이 은 은 은 요 은 많이 하나요 ? <EOS>

> 한국인이에요.
> 한국인들은 매콤한 음식을 좋아하지요. 호텔 정문 바로 건너편 오른쪽에 멕시칸 푸드 전문점이 있어요.
= 정문에서 바로 보입니까?
< 이 지역 에서는 어디 로 인가요 ? <EOS>

> 그리고 파도가 치는 바다가 아니기 때문에 뱃멀미를 하실 일도 없을 겁니다.
> 듣고 보니 그렇군요. 배를 타고 가야겠어요.
= 큰 배는 30분에 한 대씩 출발하고요 노를 저어 가는 작은 배는 바로 출발합니다.
< 어느 분 대 의 30 분 가 죠 ? <EOS>

> 네 알겠습니다. 괜찮습니까?
> 너무 어지러워 조금 쉬어야겠습니다.
= 얼마 동안 먹지 못했습니까?
< 그럼 저기 은 은 은 요 . 오 . <EOS>

> 네 붉은색 건물 2층으로 올라가셔서 신청하면 됩니다.
> 신청하면 가이드가 바로 나오나요?
= 2시간 이내로 가이드가 바로 나옵니다.
< 안쪽 서 20 분 들 고 가 면 <EOS>

> 여기 모자가 나무에 걸렸어요.
> 어떻게 된 일이죠?
= 바람이 너무 강해서 갑자기 날아가버렸어요.
< 바람 이 너무 강해서 갑자기 날아가 버렸 어요 . <EOS>

> 1인당 1만 2천 원입니다.
> 그럼 조식 서비스도 추가해주세요.
= 네 조식은 지하 1층에 뷔페식으로 마련됩니다.
< 죄송합 니다 . 저희 가 지하 1 층 은 어떻 게 해 드릴 까 요 ? <EO

< 오늘 이 곳 이 곳 이 곳 적 은 추천해 주 이 곳 이 지역 의 다음 에 있으 이 있으 니 이 곳 기념품 . 을 주 세요 . <EOS>

> 네 여기 있습니다.
> 확인되었습니다. 여기 환불 금액입니다.
= 감사합니다.
< 그럼 드신 를 오늘 결제 해 드리 겠 습니다 . <EOS>

> 그러니깐 이렇게 멀리까지 먹으러 왔죠.
> 감사합니다. 맛있다고 하시니 감사하네요.
= 순대 2인분만 더 싸주실래요?
< 네 . <EOS>

> 저는 스포츠를 좋아하는데 이 지역에서 볼 수 있는 스포츠는 어떤 것이 있습니까?
> 우리 지역에는 축구팀 야구팀 배구팀이 존재합니다.
= 저는 야구를 보고 싶은데 요즘 볼 수 있습니까?
< 다른 지역 가 있다 나요 ? ? 가 오는 그 곳 에서 얼마나 걸릴 까 요 ? <EOS>

> 네 박물관에 소장 중인 국보 백제금동대향로를 만든 것이에요. 백제금동대향로는 박물관을 대표하는 유물이라고 할 수 있죠.
> 역시 정말 아름답군요.
= 그렇죠? 많은 관광객분들께서도 기념품으로 사가세요.
< 즐거운 하루 만 이 그림 입니 . 총 가격 이 그 쪽 입니 다 . 분 들 그 은 세요 . <EOS>

> 그러니까요? 한번 살펴봐야 할 것 같습니다.
> 저는 좀 시간이 빠듯한데.
= 죄송합니다. 여기서 다른 차를 잡아 보셔야 할 것 같습니다.
< 그럼 5 쪽 으로 갈 테 을 변경 되었 습니다 . <EOS>

> 가는 길에 휴게소는 2번 들른다고 합니다.
> 먹는 것을 사가지고 타는 것이 좋겠군요.
= 휴게소의 물가가 비싸서 그러는 것이 좋습니다.
< 휴게소 의 물가 가 가 가 비싸서 그러 는 것 같아 요 . <EOS>

> 무슨 허브인데요?
> 레몬 밤입니다.
= 레몬 향이 나는 것 같네요.
< 싱글 은 빵 이 가 요 보세 요 . <EOS>

> 왜 물고기를 2마리까지만 가져갈 수 있는 거죠?
> 이곳은 한 사람당 두 마리의 물고기까지만 가져갈 수 있도록 법으로 정해져 있습니다.
= 그럼 잡은 물고기를 통에 넣지 않고 바로바로 바다에 풀어줘도 되겠네요

< 2 시 에 4 분 까지 가 있 . . <EOS>

> 죄송합니다. 또 다른 필요한 게 있으신가요?
> 수프가 너무 짭니다. 그리고 스테이크 고기는 너무 익었어요.
= 저희가 음식을 다시 준비해 드리겠어요.
< 괜찮 어요 . . . 어요 . . . . . 은 . . . 을 수 어요 ? . . <EOS>

> 한국에서 쌀로 만드는 발효주입니다. 한국산 와인이라고 할 수 있어요.
> 쓰지 않을까요?
= 아닙니다. 막걸리에 사이다를 섞어 마시면 이 또한 별미입니다.
< 그럼 은 할 요 지금 할 듯 해 . 서 해 요 . 요 . 배 가 납 니다 . <EOS>

> 어느 쪽으로 가야하나요?
> 내리막길을 따라서 가시다가 육교 앞에서 우회전하세요.
= 아 거기가면 모든 지하철노선을 이용할 수 있나요?
< 그 곳 이 계속 계속 계속 계속 됩 니다 . <EOS>

> Cash Back이 무엇입니까?
> 현금을 인출하는 것입니다. 현금이 필요하시면 드리는 것입니다.
= 그럼 40만 원으로 하겠습니다.
< 네 감사합 니다 . 그럼 제 가 가져 다 드리 겠 습니다 . <EOS>

> 호텔 지하의 마트에서 국제전화카드를 판매하고 있습니다.
> 그렇군요. 감사합니다.
= 공중전화에는 동전을 사용하셔야 하므로 1층 카운터에서 문의하시기 바랍니다.
< 네 . . . 그럼 2 열차 . <EOS>

> 싱글룸 하나 예약되어 있습니다.
> 예약이 되어 있지 않습니다.
= 공항에서 분명히 예약했습니다.
< 꼭 할 수 . 수 할 수 ? 할 <EOS>

> 눈이 그치면 다시 운행이 재개될 가능성이 있으니 잠시만 기다리십시오.
> 눈이 언제 그치나요?
= 일기예보에는 오늘 저녁에 그친다고 합니다.
< 일기예보 에는 오늘 에 에 에 에 . . <EOS>

> 서점이군요. 지갑의 외관을 알려 주시겠습니까?
> 갈색 가죽 반지갑이고 동전을 넣는 부분이 조금 착색되어 있어요.
= 지갑에 돈은 얼마나 들어 있었습니까?
< 그럼 따뜻한 가까운 가요 ? 가 어디 요 ? <EOS>

> 안내 방송에서는 구름이 많

< 그럼 다시 한번 다시 확인해서 드리 겠 습니다 . <EOS>

> 예.
> 호텔키를 분실했습니다.
= 여권을 보여주시겠습니까?
< 저 는 오늘 처음 . <EOS>

> 신용 카드는 같은 것으로 하시겠어요?
> 네.
= 예약이 됐습니다. 여기 예약번호입니다.
< 얼마 입 니까 ? <EOS>

> 그 시간에는 점심 식사와 함께 할 수 있는 관광이 준비되어 있습니다.
> 그렇습니까?
= 점심식사 패키지로 예약하시겠습니까?
< 예 요 에 바로 있 알 겠 . . <EOS>

> 저도 지금 그 버스를 기다리고 있는데요 정류장에 버스 스케쥴이 나와있으니 같이 확인해 보시죠.
> 알겠습니다.
= 막차 시간이 11시인데 벌써 11시 30분이니 아무래도 막차를 놓친 것 같네요.
< 더 필요한 것 이 없으 신 것 되 죠 . <EOS>

> 아 여기 사진에 숫자 보이시죠? 그냥 숫자로 얘기하시면 됩니다.
> 그렇군요. 그럼. 2번 3개 주세요.
= 콜라 사이즈는 어떻게 하시겠어요?
< 지금 알아보니 없으 신 올 없으 신 가요 ? <EOS>

> 우선 물수건으로 닦으세요.
> 감사합니다.
= 받침대도 여기 있습니다.
< 받침 대도 있 습니다 . <EOS>

> 무게를 재봐야 알겠지만 보통 편지 한 통은 1500원이면 됩니다.
> 다 됐습니다. 이 편지를 부쳐주세요.
= 네. 1500원 결제해주시면 됩니다.
< 지금 바로 찍 를 이 이 . 있 . <EOS>

> 저는 일반석을 원합니다.
> 일반석은 이미 매진입니다.
= 그럼 내일 것으로 미리 예매는 할 수 있나요?
< 네 그런데 가격 이 카드 지 알 고 다시 요 습니까 습니까 오 . <EOS>

> 스테이크는 어떻게 요리해 드릴까요?
> 셋 다 중간으로 익혀 주십시오.
= 에피타이저 먼저 준비해 드리겠습니다. 즐거운 시간 되십시오.
< 네 알 겠 습니다 . <EOS>

> 네 디저트는 무엇이 있나요?
> 디저트는 녹차 커피 레몬티 탄산음료 아이스크림이 준비되어 있습니다.
= 그럼 녹차와 커피를 주세요.
< 그럼 녹차 와 커피 를 